In [13]:
import numpy as np
import pandas as pd
from IPython.display import display

In [14]:
df = pd.read_csv('../cnn2_output/predict.csv')
display(df[:50])

,fname,label,prob
0,clip_000044442.wav,no,0.999996
1,clip_0000adecb.wav,unknown,1.000000
2,clip_0000d4322.wav,unknown,0.955435
3,clip_0000fb6fe.wav,up,0.444327
4,clip_0001d1559.wav,unknown,1.000000
5,clip_0002256ed.wav,unknown,0.999595
6,clip_0002a4a1f.wav,unknown,0.962246
7,clip_0002d9b83.wav,unknown,0.999999
8,clip_000373a5b.wav,go,0.999988
9,clip_0003c7122.wav,unknown,0.996867


In [15]:
filenames = df.fname.tolist()
pred = df.label.tolist()
prob = df.prob.tolist()
print filenames[:10]
print pred[:10]
print prob[:10]

['clip_000044442.wav', 'clip_0000adecb.wav', 'clip_0000d4322.wav', 'clip_0000fb6fe.wav', 'clip_0001d1559.wav', 'clip_0002256ed.wav', 'clip_0002a4a1f.wav', 'clip_0002d9b83.wav', 'clip_000373a5b.wav', 'clip_0003c7122.wav']
['no', 'unknown', 'unknown', 'up', 'unknown', 'unknown', 'unknown', 'unknown', 'go', 'unknown']
[0.99999624490700012, 1.0, 0.95543515682199998, 0.44432734779399996, 1.0, 0.99959474802000003, 0.96224594116200002, 0.999999403954, 0.99998831748999994, 0.99686706047499996]


In [16]:
Hyper_ratio= 0.7

In [17]:
dic = {'fname':[], 'label':[]}
for i in range(len(filenames)):
    if prob[i] < Hyper_ratio:
        pred[i] = 'unknown'
dic['fname'] = filenames
dic['label'] = pred
df_convert = pd.DataFrame(dic)
df_convert.set_index('fname', inplace = True)
display(df_convert[:10])

,label
fname,
clip_000044442.wav,no
clip_0000adecb.wav,unknown
clip_0000d4322.wav,unknown
clip_0000fb6fe.wav,unknown
clip_0001d1559.wav,unknown
clip_0002256ed.wav,unknown
clip_0002a4a1f.wav,unknown
clip_0002d9b83.wav,unknown
clip_000373a5b.wav,go


In [18]:
df_convert.to_csv('../cnn2_output/test.csv')